In [1]:
'''
SCRIPT PER PROVARE configurazioni SVM
'''


import codecs
import csv
import argparse
import os
import numpy as np

from time import time
from dill import dill
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import cross_validation, svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree.tree import DecisionTreeClassifier
from nltk.corpus import stopwords

# modulo per confrontare il risultato delle regex
# richiede entrambi i db già etichettati
import tortellino_parmigiano 
import crea_db


import evaluation_report # stampa confusion_matrix e altro
from text_tokenizer import get_tokenizer 

/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# list of possible algorithms
algos = ['svm-lin', 'svm-rbf', 'nb', 'decisiontree', 'randomforest', 'regex', 'svm-multi']
    # svm-rbf might not apply to text problems: https://charlesmartin14.wordpress.com/2012/02/06/kernels_part_1/




DATA_FOLDER = "/home/backup/dati_e_csv/" # DATI


# ci assicuriamo siano presenti i DB
training_db_name = crea_db.build_training(OUTPUT=DATA_FOLDER)
test_db_name = crea_db.build_test(OUTPUT=DATA_FOLDER) ################################

texts = list()
labels = list()
names= []

# load training dataset # possiamo usare sqlite
with codecs.open(os.path.join(DATA_FOLDER, training_db_name), 'r', errors='ignore') as file:
    reader = csv.reader(file, delimiter=';')
    etichette = next(reader)
    for row in reader:
        # load documents
        #print('Nome {}, etichette {}, conversazione{}'.format(row[10], row[:10], row[11]+row[12]))
        #raise SystemExit
        unito = row[11]+row[12].replace('#O:', '').replace('#C:', '') #concatena il testo cliente con operatore
        texts.append(unito)
        # load row name
        names.append(row[10])
        # load labels
        labels.append(list(map(lambda x: int(x), row[:10])))
Y = np.array(labels)
print(len(texts), len(labels))
# tokenization function # possiamo usare anche solo un canale cliente/operatore
analyzer = get_tokenizer(stopwords=stopwords.words('italian'), #word_ngrams=[1],
                         additional_extractors=None)



algo = OneVsRestClassifier(SVC(kernel='linear', max_iter=5000), n_jobs=1)
#algo = OneVsRestClassifier(LinearSVC(C=9), n_jobs=2)
#print('Multilabel ', algo.multilabel_)

# pipeline definition
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer='word', max_df=0.6,min_df=0.01, max_features=1500)),# feature extraction
    ('tfidf', TfidfTransformer(use_idf=True)), # weighting
    #('chi2', SelectKBest(chi2, k=k)), # feature selection

########### Per il chi2 multilabel
# selected_features = [] 
# for label in labels:
#     selector = SelectKBest(chi2, k='all')
#     selector.fit(X, Y[label])
#     selected_features.append(list(selector.scores_))

# // MeanCS 
# selected_features = np.mean(selected_features, axis=0) > threshold
# // MaxCS
# selected_features = np.max(selected_features, axis=0) > threshold

    ('clf', algo) # learning algorithm
])

#parameters = {#'vect__ngram_range': ((1, 1),(1, 2)),
#              'vect__max_df': (0.5,0.75,1.0),
#              'vect__max_features': (None,50,200,500,1000), # TO-DO: bisogna stemmare le features
#              'tfidf__use_idf': (True,False),
#              #'tfidf__smooth_idf': (True,False)
#              'clf__estimator__C': (0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1),
              #'clf__estimator__penalty': ('l1','l2')
#             } # vuole min e max --> facciamo unigrammi e bigrammi



#gs = GridSearchCV(pipeline, parameters, scoring='f1_macro', n_jobs=-1)


#print('Pipeline creata, avvio il modello')

print(algo)

Training sqlite e csv presenti!
Carico training_regex.csv
Test sqlite e csv presenti!
Carico test_manina
20706 20706
OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=5000, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=1)


Grid Search celle larghe:

Best score: 0.464

Best parameters set:

clf__estimator__C: 1

tfidf__use_idf: True

vect__max_df: 0.5

vect__max_features: 1000

GRID SEARCH 2:

done in 822.022s
Best score: 0.488
Best parameters set:
	clf__estimator__C: 1.5
	vect__max_df: 0.5
	vect__max_features: 1500
    
GRID SEARCH 3:
done in 935.240s
Best score: 0.480
Best parameters set:
	clf__estimator__C: 10
	vect__max_features: 2500
	vect__min_df: 0
    
    
GS 4:
Best score: 0.488
Best parameters set:
	clf__estimator__C: 9
	vect__max_df: 0.5
	vect__max_features: 1500
	vect__min_df: 0


In [6]:
# Grid search da saltare

print('grid search in action...')
t0 = time()
gs.fit(texts, Y)
print("done in %0.3fs" % (time() - t0))
print("Best score: %0.3f" % gs.best_score_)
print("Best parameters set:")
best_parameters = gs.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

grid search in action...


/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set t

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]done in 1877.411s
Best score: 0.488
Best parameters set:
	clf__estimator__C: 9
	vect__max_df: 0.5
	vect__max_features: 1500
	vect__min_df: 0


In [3]:
# controlla se esiste il modello per evitare di ricrearlo ogni volta

if False: #os.path.exists(os.path.join(DATA_FOLDER, 'OneVsRestClassifier')):
    with open(os.path.join(DATA_FOLDER, 'OneVsRestClassifier'), mode='rb') as modelfile:
        model = dill.load(modelfile)

else:
    model = pipeline.fit(texts, Y)
    print('Modello creato, verifico il test set')


    with open(os.path.join(DATA_FOLDER, 'OneVsRestClassifier'), mode='wb') as modelfile:
        dill.dump(model, modelfile)

/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, 

Modello creato, verifico il test set


In [9]:
# importo il test set per la valutazione del modello
unito_test = []
names_test = []
test_texts = list()
test_labels = list()
#raise SystemExit

# load test data
# carica il test set
with codecs.open(os.path.join(DATA_FOLDER, test_db_name), 'r', errors='ignore') as file:
    reader = csv.reader(file, delimiter=';')
    next(reader)
    print(etichette)
    for row in reader:
        # load documents
        test_texts.append((row[1]+row[2]).replace('O:', '').replace('C:', ''))
        # load names
        names_test.append(row[0])
        # load labels
        test_labels.append(list(map(lambda x: int(x),row[-10:])))

['attivazione', 'canone rai', 'cessazione', 'contratto', 'credito', 'domiciliazione', 'fatturazione', 'gr-invito a chiamare - focus', 'non di competenza', 'richiamate', 'xml name', 'testo operatore', 'testo cliente']


In [12]:
Y_test = np.array(test_labels)
# apply model to data
predicted_labels = model.predict(test_texts) #np_array

# print evaluation report
evaluation_report.print_evaluation(Y_test, predicted_labels)
for i in range(predicted_labels.shape[1]):
    print(etichette[i], np.sum(predicted_labels[:,i]))
print('Decision function: ')

scores = model.decision_function(test_texts) #np_array
print('\n')
print('accuracy', accuracy_score(Y_test, predicted_labels))
print('precision micro',precision_score(Y_test, predicted_labels, average='micro') )
print('recall micro',recall_score(Y_test, predicted_labels, average='micro'))
print('f1 micro', f1_score(Y_test, predicted_labels, average='micro'))
print('precision macro', precision_score(Y_test, predicted_labels, average='macro') )
print('recall macro', recall_score(Y_test, predicted_labels, average='macro'))
print('f1 macro', f1_score(Y_test, predicted_labels, average='macro'))



[[0 1 0 1 0 0 1 0 0 0]
 [0 0 0 0 1 0 1 0 0 0]
 [1 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]]
[[0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
Confusion matrix attivazione
[[343   8]
 [ 61  12]]
Confusion matrix canone rai
[[360   1]
 [ 54   9]]
Confusion matrix cessazione
[[414   0]
 [ 10   0]]
Confusion matrix contratto
[[331  19]
 [ 44  30]]
Confusion matrix credito
[[168  94]
 [ 15 147]]
Confusion matrix domiciliazione
[[400   3]
 [ 20   1]]
Confusion matrix fatturazione
[[274  26]
 [ 96  28]]
Confusion matrix gr-invito a chiamare - focus
[[216 185]
 [  5  18]]
Confusion matrix non di competenza
[[373   1]
 [ 47   3]]
Confusion matrix richiamate
[[414   7]
 [  3   0]]
                              precision    recall  f1-score   support

                 attivazione       0.60      0.16      0.26        73
                  canone rai       0.90      0.14      0.25        63
                  c

/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)




accuracy 0.143867924528
precision micro 0.418918918919
recall micro 0.41127694859
f1 micro 0.415062761506
precision macro 0.432939187344
recall macro 0.27360877122
f1 macro 0.238698316071


/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [47]:
# score della cat credito 
cred_score = scores[:,4]
cred_pred = predicted_labels[:,4]
#print(len(cred_score), cred_pred)

positive = cred_score[cred_pred == 1]
neg = cred_score[cred_pred == 0]
#print(positive)
print('media score positivi predetti:   ', positive.mean())
print('media score negativi predetti:   ', neg.mean())
print('std score positivi predetti:     ', positive.std())
print('std score negativi predetti:     ', neg.std())

media score positivi predetti:    0.769703031016
media score negativi predetti:    -1.16050984824
std score positivi predetti:    0.688920536459
std score negativi predetti:    0.579833964186


In [49]:
# confronto gli score dei doc annotati a mano con quelli assegnati dal modello
cred_score = scores[:,4]
cred_ammano = Y_test[:,4]
#print(len(cred_score), cred_ammano)

positive_ammano = cred_score[cred_ammano == 1]
neg_ammano = cred_score[cred_ammano == 0]
print(sum(cred_ammano), len(positive_ammano))
print('media score positivi:   ' , positive_ammano.mean())
print('media score negativi:   ', neg_ammano.mean())
print('std score positivi:     ', positive_ammano.std())
print('std score negativi:     ', neg_ammano.std())

156 156
media score positivi:    -0.333678927128
media score negativi:    -1.2992404636
std score positivi:    0.85045030223
std score negativi:    0.637909131329


ImportError: No module named 'sklearn.model_selection'